<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# To Check Solar's ability

In [19]:
!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [20]:
# Additional Contents : https://wikidocs.net/253106 

In [18]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()
file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()
file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [117]:
# read test_samples.csv file

import pandas as pd
import os

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers


now_topic = 'law'

prompts, answers = read_data(os.path.join(data_path, f'mmlupro_test_{now_topic}.csv'))

## 4. Prompt engineering

In [119]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    ---
    Please provide most correct answer.
    The answer format is as follow.

    ---
    Question: "Which of the following is most likely to be found in a library?
    (A) A basketball
    (B) A book
    (C) A painting
    (D) A computer
    (E) A refrigerator"
    Answer: (B) A book
    ---
    
    ---
    Question: {question}
    Answer: 
    ---

    '''

)
chain = prompt_template | llm

responses1 = []

for prompt in prompts:
    response = chain.invoke({"question": prompt})
    responses1.append(response.content)

In [72]:
from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    Please provide most correct answer. Let's think step by step.
    
    The Answer format is as follow (do not explain) : 
    Answer: (D) keyword.
    ---
    
    Question: {question}
    Answer :
    ---
        

    '''

)
chain = prompt_template | llm

responses2 = []

for prompt in prompts:
    response = chain.invoke({"question": prompt})
    responses2.append(response.content)

In [120]:
for i in responses1:
    print(i)
    print('-')


Answer: (D) Robbery, because he used force in leaving with the purse.
-
Answer: (C) No, because the corporation was not granted use-and-derivative-use immunity.
-
Answer: (G) It unreasonably limits the means by which a candidate can campaign, infringing on democratic practices.
-
Answer: (I) The defendant was justified in using reasonable force in the woman's defense, since he reasonably believed she was in immediate danger of unlawful bodily harm from the man.
-
Answer: (F) not guilty due to lack of intent.
-
Answer: (F) guilty, if his advances were unwelcome.
-
Answer: (G) permissive.
-
Answer: (B) She must produce the MRI scan as it is a tangible evidence and can be examined by the court.
-
(B) discharged because more than 175 days passed between his release from jail and the filing of the motion to discharge.
-
The correct answer is (F) inadmissible, because it is hearsay evidence. The court officer's testimony is considered hearsay evidence, which is generally not admissible in co

In [121]:
#answers = answers[::100]

## Check Accuracy

In [122]:
try : del [[wrongdf]]
except : pass
wrongdf = pd.DataFrame(columns=['questionNum', 'answer', '1st_pred'])

In [123]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

In [124]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, responses1)):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":"-"}
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":None}
        
acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("1st wrong:", wrong)
wrongdf.loc[len(wrongdf)] = {"questionNum":"-", "answer" : "-", "1st_pred":acc}

----------
Answer: (D) Robbery, because he used force in leaving with the purse.
idx: 0 | generated answer: D, answer: D
----------
Answer: (C) No, because the corporation was not granted use-and-derivative-use immunity.
idx: 1 | generated answer: C, answer: B
----------
Answer: (G) It unreasonably limits the means by which a candidate can campaign, infringing on democratic practices.
idx: 2 | generated answer: G, answer: F
----------
Answer: (I) The defendant was justified in using reasonable force in the woman's defense, since he reasonably believed she was in immediate danger of unlawful bodily harm from the man.
idx: 3 | generated answer: I, answer: A
----------
Answer: (F) not guilty due to lack of intent.
idx: 4 | generated answer: F, answer: E
----------
Answer: (F) guilty, if his advances were unwelcome.
idx: 5 | generated answer: F, answer: C
----------
Answer: (G) permissive.
idx: 6 | generated answer: G, answer: G
----------
Answer: (B) She must produce the MRI scan as it is

In [125]:
wrongdf.to_csv(results_path+f"solarmini_1shot_mmlupro_{now_topic}.csv", index=False)